## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Чтобы ответить на поставленные вопросы, мне будут нужны столбцы: ['children', 'family_status', 'total_income', 'debt', purpose']. Именно из них будет состоять таблица, с которой я буду работать, так как при предварительном просмотре полная версия занимает слишком много места на маленьком экране.

Есть недочеты в след. столбцах:

1. Кол-во детей - значения -1 и 20. Возможно, информация была либо некорректная заведена изначально при перенесении ее из анкеты, либо при выгрузке (но тут не уверена, так как мне непонятно, как именно могли появиться такие значения;
2. Ежемесячный доход - пропущено 10% значений, что достаточно много.
3. Стаж работы - положительные только у безработных и пенсионеров, причем их стаж 900+ лет, а отрицательный у всех остальных. Возможно, в систему внеслось что-то не так, либо у них прописан стаж не в днях, что закладывалось изначально.
4. Пол клиента - непонятное XNA для 1 человека.
5. Разный регистр в столбце с образованием.
6. Большое количество однотипных целей кредита.

### Шаг 2. Предобработка данных

### Обработка пропусков

Даны 4 вопроса, на которые обязательно необходимо ответить в исследовании:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

    Для меня также будет интересно посмотреть зависимость между типом занятости и возвратом кредита в срок.

Для ответа на первый вопрос, необходимо будет почистить столбец с количеством детей, на данный момент у кого-то -1 и 20 детей, не сильно в это верится.

Данные для ответа на второй вопрос все есть и они в хорошем состоянии.

Чтобы ответить на 3й вопрос, убрать пропуски в столбце дохода. Я собираюсь применить два варианта для сравнения:
- Замена пропущенных данных на среднее значение по каждому типу занятости.
- Удаление всех строк с пропущенными значениями, пожертвовав 10% данных.

Для 4го вопроса нужны будут доподнительные манипуляции.

- В датафрейме data NaN будут заменены на среднее значение по группам. С этим же датафреймом я буду проводить остальные манипуляции и действия.
- В датафрейме new_data я удалю строки с пропущенными значениями.

In [3]:
# очень некрасивые сейчас данные в столбце "образование", 
# приведем их к единому регистру.
data['education'] = data['education'].str.lower() 

# новый датарейм new_data для второго варианта избавления отпропусков - удаление строк с пропусками.
new_data = data[['children', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']] 
new_data.head(15)

,children,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# удаление строк с пропущенными значениями
new_data.dropna(subset = ['total_income'], inplace = True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 8 columns):
children            19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 1.3+ MB


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


В data я заменяю пропущенные данные на среднее значение по каждому типу занятости. 

Для этого понадобится табличка, в которой будут указаны средние значения по типу занятости. Далее обращаюсь к ней для подстановки недостающих элементов.

In [10]:
mean_table_income = data.groupby('income_type').agg({'total_income':'mean'})

for income_type in mean_table_income.index:
    data.loc[data['income_type'] == income_type, 'total_income'] = data.loc[
        data['income_type'] == income_type, 'total_income'].fillna(mean_table_income.loc[income_type, 'total_income'])

data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Я создала табличку со средними значения, чтобы из нее при помощи цикла заполнить пропуски в столбце стажа работы. Так как данные этого столбца далее не будут использованы, то не играет большой роли, каки именно методом они будут удалены.

In [12]:
mean_table = data.groupby('income_type').agg({'days_employed':'mean'})

for inc_type in mean_table.index:
    data.loc[data['income_type'] == inc_type, 'days_employed'] = data.loc[
        data['income_type'] == inc_type, 'days_employed'].fillna(mean_table.loc[inc_type, 'days_employed'])


Исправим ошибку в поле клиента, заменив XNA на M. Так как это всего лишь для 1 человека то будет это М или F, также не сильно важно.

In [15]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'M'
data['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

Исправим ошибки в количестве детей. Я предполагаю, что ошибки возникли при перенесении информации в базу рукой человека, поэтому -1 = 1 и 20 = 2 ребенка в семье.

In [16]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

1. Я заменила пропуски в столбце стажа путем подставновки среднего значения по типу занятости.
2. Произвела замену пола для одного человека с XNA на М.
3. Замена в поле дети с количества -1 на 1, и с 20 на 2.
4. Я заменила пропуски в столбце ежемесячный доход двумя способами, как и хотела: путем подставновки среднего значения по типу занятости и удалением строк с NaN.

### Замена типа данных

In [17]:
data['days_employed'] = data['days_employed'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Для выполнения данного задания был использован метод .astype() для замены типа данных в столбце стажа, в таком формате легче просматривать данные, не сбивает огромное количество цифр.

### Обработка дубликатов

In [18]:
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Для поиска грубых совпадений использован метод .duplicated(), для избавления от них - метод drop_duplicated(). Учитывая, что дубликатов было 71, причина их появления скорее всего в том, что они были занесены в базу дважды из-за невнимательности сотрудников.

### Лемматизация

In [19]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_lemmas'] = data['purpose'].apply(m.lemmatize)

# Считаем количество упоминаний каждой леммы 
# и сортируем по частоте упоминаемости
from collections import Counter
count_purpose_lemmas = Counter(data['purpose_lemmas'].sum())
count_purpose_lemmas.most_common()

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

In [20]:
# Создание добавочного столца, где будут записаны цели кредита одним словом, 
# заменяя огромное количество повторяющихся вариантов, но в разными словами

def purpose_search(purpose_lemmas):
    if ('недвижимость' in purpose_lemmas) or ('жилье' in purpose_lemmas):
        return 'Недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'Автомобиль'
    if 'образование' in purpose_lemmas:
        return 'Образование'
    return 'Свадьба'
    
data['new_purpose'] = data['purpose_lemmas'].apply(purpose_search)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,new_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",Недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",Недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",Образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",Свадьба


### Вывод

Отдельным столбцом вынесла леммы целей кредита, чтобы избавиться от похожих мелких целей, вроде "покупка недвижимости", "покупка жилой недвижимости", "покупка коммерческой недвижимости", и присвоить в итоге всем общие цели кредита.

Таким образом в столбце new_purpose получилось всего 4 большие группы целей, по которым далее будет легче категоризировать данные: Недвижимость, Образование, Автомобиль, Свадьба. Для этого было использовано:
1. Процесс лемматизации, импорт библиотеки pymystem3.
2. Метод apply(), который применял функцию к каждой строке, тем самым создав отдельный столбец с леммами.
3. Отсортировав леммы по частоте упоминаемости, я выявила 4 частоупоминаемые цели: Недвижимость (объеденено с жильем), Образование, Свадьба, Автомобиль.
4. Присвоила каждому клиенту его вид цели, выделила их в отдельный столбец.

### Категоризация данных

In [21]:
# Создание отдельного столбца-индикатора, 
# который говорит о наличии детей (в любом количестве) или же об их отсутствии

def parental_status(children):
    if children == 0:
        return 0
    return 1

data['children_id'] = data['children'].apply(parental_status)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,new_purpose,children_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",Недвижимость,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Автомобиль,1
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",Недвижимость,0
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",Образование,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",Свадьба,0


#### Категоризация по ежемесячному доходу

Так как я решила избавиться от пустых значений в колонке дохода двумя способами, то необходимо провести категоризацию  в двух датафреймах.

In [22]:
# Датафрейм, где заменяла пропущенные значения на средние.

data['total_income'] = data['total_income'].astype('int')
data.sort_values(by='total_income')
data['total_income'].describe()

def income_classification (total_income):
    if total_income < 1.076230e+05:
        return 'критичный'
    if 1.076230e+05 < total_income < 1.518870e+05:
        return 'низкий'
    if 1.518870e+05 < total_income < 2.024170e+05:
        return 'средний'
    return 'высокий'

In [64]:
data['total_income_category'] = data['total_income'].apply(income_classification)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,new_purpose,children_id,total_income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Недвижимость,1,высокий
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Автомобиль,1,низкий
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Недвижимость,0,низкий
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Образование,1,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Свадьба,0,средний


In [73]:
# Датафрейм, где удалила строки с пропущенными значениями.

new_data['total_income'] = new_data['total_income'].astype('int')
new_data.sort_values(by='total_income')
new_data['total_income'].describe()

def income_classification2 (total_income):
    if total_income < 1.076230e+05:
        return 'критичный'
    if 1.076230e+05 < total_income < 1.518870e+05:
        return 'низкий'
    if 1.518870e+05 < total_income < 2.024170e+05:
        return 'средний'
    return 'высокий'

In [75]:
new_data['total_income_category'] = new_data['total_income'].apply(income_classification2)
new_data.head()

,children,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высокий
1,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,низкий
2,0,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,низкий
3,3,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,высокий
4,0,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний


### Вывод

Используя метод describe(), нашла значение трех квартилей, при помощи которых разделла всех клиентов на 4 категории по уровню дохода: критичный, низкий, средний, высокий, поместила их в отдельный столбец, чтобы посмотреть сколько по каждой категории должников и сколько добросовестных плательщиков.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [65]:
# Создание сводной таблицы, где будет информация о том, 
# у скольких человек с детьми и без детей был долг по кредиту

pivot_table_children = data.pivot_table(index = 'children_id', columns = 'debt', values='income_type', aggfunc = 'count')


# В сводную таблицу добавлена сумма людей без детей и с детьми 
# и посчитан процент людей, у которых был долг перед банком

pivot_table_children['total'] = pivot_table_children[0] + pivot_table_children[1]
pivot_table_children['debt %'] = pivot_table_children[1] / pivot_table_children['total'] * 100
pivot_table_children

debt,0,1,total,debt %
children_id,,,,
0,13028,1063,14091,7.543822
1,6685,678,7363,9.208203


### Вывод

Процент тех людей, у кого не было долгов, ниже у людей без детей, но разница минимальна - 1,7%, поэтому я не стану утверждать, что наличие или отсутствие детей сильно влияет на возможность и желание вернуть кредит в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [78]:
# Сводная таблица, где все клиенты разделены по семейному положению
# и показывается сколько в каждой группе должноков и ответственных плательщиков.

pivot_table_family_status = data.pivot_table(index='family_status', columns='debt', values='income_type', aggfunc = 'count')
pivot_table_family_status['total'] = pivot_table_family_status[0] + pivot_table_family_status[1]
pivot_table_family_status['% debt'] = pivot_table_family_status[1] / pivot_table_family_status['total'] *100
pivot_table_family_status.sort_values(by='% debt', ascending=False)

debt,0,1,total,% debt
family_status,,,,
Не женат / не замужем,2536,274,2810,9.750890
гражданский брак,3763,388,4151,9.347145
женат / замужем,11408,931,12339,7.545182
в разводе,1110,85,1195,7.112971
вдовец / вдова,896,63,959,6.569343


### Вывод

Судя по сводной таблице, исправнее всего выплачивает кредиты категория "вдовец/вдова", а самое большое количество должников в категории "не женат/не замужем", разница между ними 3,1%. Данный показатель наводит на мысль, что некая взаимосвязь существует, но навряд ли только семейное положение повлияло на возможность исправно выплачивать кредит.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [81]:
pivot_table_total_income = data.pivot_table(index='total_income_category', columns='debt', values='income_type', aggfunc='count')
pivot_table_total_income['total'] = pivot_table_total_income[0]+pivot_table_total_income[1]
pivot_table_total_income['% debt'] = pivot_table_total_income[1]/pivot_table_total_income['total'] * 100
pivot_table_total_income.sort_values(by='% debt', ascending=False)

debt,0,1,total,% debt
total_income_category,,,,
низкий,4888,475,5363,8.856983
средний,4853,464,5317,8.726726
критичный,4937,427,5364,7.960477
высокий,5035,375,5410,6.931608


In [82]:
pivot_table_total_income2 = new_data.pivot_table(index='total_income_category', columns='debt', values='income_type', aggfunc='count')
pivot_table_total_income2['total'] = pivot_table_total_income2[0]+pivot_table_total_income2[1]
pivot_table_total_income2['% debt'] = pivot_table_total_income2[1]/pivot_table_total_income2['total'] * 100
pivot_table_total_income2.sort_values(by='% debt', ascending=False)

debt,0,1,total,% debt
total_income_category,,,,
низкий,4537,440,4977,8.840667
средний,3743,359,4102,8.751828
критичный,4937,427,5364,7.960477
высокий,4563,345,4908,7.029340


### Вывод

1. Сначала про разницу между двумя подходами - она составляет всего 0.099%, поэтому в данном случае оба варианта замены пропусков (удаление строк с пропусками и замена на среднее) равносильны, можно использовать любой.
2. Касательно зависимости по сводной таблице хуже кредиты возвращают люди с низким заработком, а лучше всего долги отдают люди с высоким заработком (как неожиданно, не правда ли). Хочу только отметить, что чуть хуже, но все равно исправно платят по кредитам люди с критичным (супер низким) заработком, и вот это уже интереснее. Есть предположение, что такие люди больше болнуются за свою кредитную историю, поэтому хоть и не все, но стараются прижерживаться сроков. 
3. Разница между ними почти 2%, поэтому о сильной зависимости между уровнем дохода и возвратом кредита в срок не могу утверждать на 100%. И все же небольшая зависимость, но есть.

- Как разные цели кредита влияют на его возврат в срок?

In [83]:
pivot_table_purpose = data.pivot_table(index='new_purpose', columns='debt', values='income_type', aggfunc='count')
pivot_table_purpose['total'] = pivot_table_purpose[0]+pivot_table_purpose[1]
pivot_table_purpose['% debt'] = pivot_table_purpose[1]/pivot_table_purpose['total']*100
pivot_table_purpose.sort_values(by='% debt', ascending=False)

debt,0,1,total,% debt
new_purpose,,,,
Автомобиль,3903,403,4306,9.359034
Образование,3643,370,4013,9.220035
Свадьба,2138,186,2324,8.003442
Недвижимость,10029,782,10811,7.233373


### Вывод

Охотнее всего люди выплачивают кредит за недвижимость, нежели за автомобиль или образование. Возможно, большую роль играет устрашающие сроки выплаты (20-40 лет это не шутки), тем более процент переплаты итак огромный, зачем еще усугублять ситуацию несвоевременным внесением платежа.

- Как тип занятости влияет на возврат кредита в срок?

In [80]:
pivot_table_income_type = data.pivot_table(index='income_type', columns='debt', values='purpose', aggfunc='count')
pivot_table_income_type['total'] = pivot_table_income_type[0]+pivot_table_income_type[1]
pivot_table_income_type['% debt'] = pivot_table_income_type[1]/pivot_table_income_type['total']*100
pivot_table_income_type.sort_values(by='% debt', ascending=False)

debt,0,1,total,% debt
income_type,,,,
безработный,1.0,1.0,2.0,50.000000
сотрудник,10023.0,1061.0,11084.0,9.572357
компаньон,4702.0,376.0,5078.0,7.404490
госслужащий,1371.0,86.0,1457.0,5.902539
пенсионер,3613.0,216.0,3829.0,5.641160
в декрете,NaN,1.0,NaN,NaN
предприниматель,2.0,NaN,NaN,NaN
студент,1.0,NaN,NaN,NaN


### Вывод

В самой таблице мало людей в категориях "студент", "предприниматель", "безработный" и "в декрете", поэтому оценивать их наравле с остальными категориями не умеет смысла. 
Однако видно, что на удивление пенсионеры самые дисциплинированные по части возврата кредита, а заядлыми должниками являются рядовые сотрудники, разница почти 4% и это пока самый большой разброс среди сравниваемых категорий.

### Шаг 4. Общий вывод

Предоставленные данные были с пропусками, дубликатами, описками (скорее всего) и прочим мусором, который мог появиться как из-за человеческой ошибки, так и из-за проблем при выгрузке. Весь мусор был почищен. Было много вопросов: почему стаж отрицательный или равен 900+ лет, как пенсионеры могут зарабатывать по 200 000 рублей в месяц. С этими вопросами в реальной жизни я скорее пришла бы к тому отделу, который передал мне эти данные для анализа.

В ходе данной проектной работы я пришла к следующим выводам, отвечая на поставленные вопросы:
1. Должников с детьми на 1,7% больше, чем должников без детей. Исходя из этого, полагаю, что наличие или отсутствие детей не сильно влияет на платежеспособность клиентов.
2. Вдовцы и вдовы показывают более ответственное отношение к выплатам по кредиту нежели те, кто не состоял и не состоит в браке. Разница между ними больше 3%, возможно, и существует некая зависимость от семейного положения, но она крайне мала.
3. Оба подхода дали практически идентичные результаты. Все вполне предсказуемо - люди с высоким доходом лучше выплачивают кредиты (6,9% должников), люди с низким заработком гораздо хуже (8,8% должников). Было сюрпризом, что клиенты с критично малым заработком очень ответственно относятся к кредиту, возможно, из-за страха заработать плохую кредитную историю.
4. Интересным был результат анализа зависимости выплаты кредита от его цели. Люди более сознательно подходят к выплате кредита за недвижимость (всего 7% должников). Вполне логично, так как им же хуже, если они просрочат платежи, суммы переплат будут гораздо выше, чем за автомобиль (9% должников). Здесь разница также не сильно велика, поэтому и зависимость выплат от целей не чень весома - никому не хочется платить пени за просрочку.
5. Результаты анализа моего предположения, что тип занятости влияет на возврат кредита в срок оказались интересными: пенсионеры оказались более сознательными по части выплаты кредита в отличии от обычных сотрудников, разница между ними составила почти 4%. Интересно, но настолько ли весомо.